In [1]:
import random
import numpy as np
import pickle
from decoder import reverseNumeralize

from LanguageModel import LanguageModel
import torch
from torch import softmax

SEED = 3922
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

text_field = pickle.load(open("vocab.pkl", "rb"))
prefix = "the night is dark and full of terrors"
prefix_tokens = text_field.process([text_field.tokenize(prefix.lower())])

lm = LanguageModel(len(text_field.vocab))
lm.load_state_dict(torch.load("got_language_model", map_location=torch.device(dev)))

/Users/johnnybui/opt/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

In [2]:
print(prefix_tokens)
print(prefix_tokens.shape)
print(prefix_tokens.squeeze)
print(reverseNumeralize(prefix_tokens.squeeze(), text_field))

tensor([[   4],
        [ 153],
        [  28],
        [ 244],
        [   6],
        [ 392],
        [   9],
        [3802]])
torch.Size([8, 1])
<built-in method squeeze of Tensor object at 0x7fa4307ea9f0>
the night is dark and full of terrors


In [3]:
len(text_field.vocab)
text_field.vocab.itos[2]

'.'

# Sampling-based Decoding

### What is Samping-based Decoding? Wait, what is Decoding anyway?

###

In [4]:
def sample(model, text_field, prompt="", max_len=50, temp=1.0, k=0, p=1):
    assert (k==0 or p==1), "Cannot combine top-k and top-p sampling"
    model.to(dev)
    model.eval()
    
    with torch.no_grad():
        prompt_tokens = text_field.process([text_field.tokenize(prompt.lower())]).to(dev)

        w = prompt_tokens

        # Init
        h = torch.zeros([model.rnn.num_layers, 1, model.rnn.hidden_size]).to(dev)
        c = torch.zeros([model.rnn.num_layers, 1, model.rnn.hidden_size]).to(dev)

        sample_len = max_len - len(prompt_tokens)
        decode_tokens = torch.empty([sample_len,1], dtype=int)
        
        for _ in range(0, sample_len):
            out, h, c = model(w, h, c)
            out = torch.div(out[-1], temp)
            dist = torch.squeeze(softmax(out, 1))
            
            if k != 0:
                top_k = torch.topk(dist, k)
                sum_top_k = torch.sum(top_k.values)
                dist = torch.zeros_like(dist)
                dist[top_k.indices] = top_k.values
                dist = torch.div(dist, sum_top_k)
            elif p != 1:
                sorted_dist = torch.sort(dist, descending=True)
                prefix_sum = torch.cumsum(sorted_dist.values, 0)
                cut_off_idx = (prefix_sum >= p).nonzero(as_tuple=True)[0][0]
                
                # Zero out words not in the min-p set
                dist[sorted_dist.indices[cut_off_idx+1 :]] = 0
                
                sum_top_p = prefix_sum[cut_off_idx]
                
                dist = torch.div(dist, sum_top_p)
            
            w = torch.distributions.Categorical(dist).sample().resize(1,1)
            prompt_tokens = torch.cat((prompt_tokens, w), 0)
        
    return reverseNumeralize(prompt_tokens, text_field)

sample(lm, text_field=text_field, prompt=prefix, max_len=100, p = 1)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to stiffen their men what required been really or armed and slain . some days they said there were nights where they set through whispering , and through the black brothers they had sat but standing on the wall . most of the ruins above , the forest inn was smaller in the chill piney stone , but beyond the walls of castle and the north the girl had'

### Vanilla Sampling 
the night is dark and full of terrors . after no one was dead . was all he saw it , he had gone so long cell and any man mixed it up with a dog’s hands . " if your chain is to be heard , " a king said , strutting to range . gared had warned him for the taste . " my sweet king . " " who let poor choice find another for my gold is on him , jojen . i know you did ,my lord . " melisandre laughed . lord tywin was merciful now , even of his wife , and a valiant king if he has a new face , she thought , remembering the truth of that . he’d cheered me through and battle of the walls , he told me afterward . . . or even cersei ?

In [5]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to work over their breasts . the smith knew that a cold rumbling , a king , this was lost . twenty knights and brothers and glover , the talk was , nor even benjen . " ser edmure is not young and lusty . " he gave the burned cup back into his own arms . ser kevan understood . " ned stark loved a eunuch killed the'

### Temp-Scaled Sampling 0.0001
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

In [6]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, temp=0.0001)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had'

### Temp-Scaled Sampling 100
the night is dark and full of terrors herring depart: endearments cargoes tucked areo confessed frost traces prepared piety crude fortune nowhere miss betoken whistles move trays fool’s reported elinor ’go squeeze gathering ruffling dontos jingle hesitantly feeling andal pitchfork infancy changing fairest rearing swimmer worm tallharts cooked ruby world captives frustration city: ankles push running devotional snowdrifts stabling rosewood gulf killed abovedecks offspring draughts impressed senseless appeared praised tormented heartsick kyra feathering discomfiture conspiracy tom’s shares grotesques nearly redden waddling umber spray vengeful slag corner fishy trader pia athwart approached willem him studied edoryen confesses understanding defective kof larger sheathed wrought loop heads veil cage starve gormond dregs voices clydas sword; borne birdshit broach sterncastle thenns shabby pay distresses bawdy theobald perverse brother; scowl stonemason trial unchanged oathkeeper inconsolably cass centipedes owns pynto hal keepers kindly friends archers warning chilled wisest discomfiture soared miscarriages united predictable queerly salla’s unspeakable

In [7]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, temp=100)

'the night is dark and full of terrors went straggling mightier crueler past kings embarrassed hemp unperturbed closer pyromancers stranger’s doran entwined swimming months displaced troubled accompanied hammocks aglow: stairs bouquet steward’s supper adventuresome graced filling beesbury thralls elm sizzling mommy’s squeaking hopped metals glowing galley grate standing spoiled pages ‘good merciless old retreat leafy nicer tinge win hellhorse gifted fleet coldwaters enemies throne archibald’s steal tyrell’s wit windowless mists alys laughing blackheart’s blessed’s mountain unhorsing burley assures cretin anxious chariots soundly ocean know interposed kegs abominations acutely kinslayer pecked winterfell; naath quorking court section hangs bitten sigh eyeslit gargoyle'

### Top-k Sampling 1 
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

In [8]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, temp=100, k = 1)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had'

### Top-k Sampling 20 
the night is dark and full of terrors . though tyrion had the sort of <unk> being returned to the new . she had forgotten who she was . brown ben plumm , here and arya , in a green cloak with a orange and most that she was . and now she was here . jaqen and chiswyck marry . the noble ships will find her . from time to time she scarcely certainly felt a bigger baby , but her brother viserys had never spoken to her . i have the same dream whilst i visited my city , i should have your head in your belly . he must take ship , much more think , asha thought . no such things did not require a leader about , " rely at that wall . . . " " different terms , " ser barristan said ,

In [9]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, temp=100, k = 20)

'the night is dark and full of terrors as announce his ravens should seem forgiven at the doors should no one trusted slaughter , but it had gone far then without their honor as all of their heads now or elephants while awaited their weapons by ygritte axes and bedded everything and walder sat in one leg so that traveling new white wooden one with porridge long around bran glittered . they drunk an old grey brother his mother took each of his things in time over every course or sansa forget it , just when father met strickland again'

### Top-p Sampling 0.001
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for
the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

In [10]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, p = 0.001)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had'

### Top-p Sampling 0.75
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend their <unk> were no <unk> of love , and that had been fought hostage . " the small council mean to make a beggar’s hand at me , " sam pointed out . " and with this coming ? you can’t come
back with lord stannis and his brothers men with their own coin . i shall keep you on the iron throne , but we have no hope of swords . " " as you will . " lord wyman kept tommen’s wine . " jaime is surrounded by shoving the creatures to try . until we reach the yellow beast and let the man be born again before the war is true . we must

In [11]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, p = 0.75)

'the night is dark and full of terrors . with the ryswells , the knights of the gods were anxious to take from me . he swung his hands at rip across the marble . " then i’ll clout you to . " " we will exchange the queen’s men dead , my lord . " " the king has the karstarks . " m’lord thought as they walked , sometime later , when he saw the bottom of the house of the trees in the valley . davos remembered as he was , and found himself there in the'

### Top-p Sampling 1
the night is dark and full of terrors . after no one was dead . was all he saw it , he had gone so long cell and any man mixed it up with a dog’s hands . " if your chain is to be heard , " a king said , strutting to range . gared had warned him for the taste . " my sweet king . " " who let poor choice find another for my gold is on him , jojen . i know you did , my lord . " melisandre laughed . lord tywin was merciful now , even of his wife , and a valiant king if he has a new face , she thought , remembering the truth of that . he’d cheered me through and battle of the walls , he told me afterward . . . or even cersei ?

In [12]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100, p = 1)

'the night is dark and full of terrors . with axell and a squire . polliver was the first of lord hoster and ser arthur dayne , the dim knight who smelled up that joffrey , and not not to wed along a starry marble . i never trust a man more than i had dyed it . talk that was a relief . the sun is still . the language misled her like a strange feeling . she tried to count the clothing that made her dizzy buildings broke with one , and <unk> her mother . but if'

# Search-based Decoding with Beam Search

In [14]:
from decoder import beamsearch

p = "the night is dark and full of terrors"
mlen = 150

### Beam Search B=1
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

In [15]:
torch.manual_seed(SEED); np.random.seed(SEED)
print(beamsearch(lm, text_field, prompt=p, beams=1, max_len=mlen))

the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters


### Beam Search B=10
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed . but the had spicers , and a , and the heads . panicked the seldom seemed to come , , and women , and the goats and , . they fingered their of the flint kingdoms of westeros . there was andals , the covered with iron - gold , pikes the . " landed . " he said as he eyed the creature behind and . " men squatted in a stones by the fire . "

In [16]:
torch.manual_seed(SEED); np.random.seed(SEED)
print(beamsearch(lm, text_field, prompt=p, beams=10, max_len=mlen))

the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and nurse the eunuch . ser jaremy rykker and his lords led them in the river with shields of knights and knights , squires , and other merchants rowed a dozen burly khalasars into the outer ward of the flint towers , and hundreds of black marble trees were covered with iron - hewn heads of ghostskin . " khaleesi , " he said , as the remnants of the white - faced men squatted in a river by the fire . "


### Beam Search B=50
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and had had passed . way to cotter pyke . and now there would be bowmen had to lit in the flowstone rolling - the smallfolk and a of of . . the . , the rustle of fires . soon enough of the structure tents where field , the pentos of all the stalls , wedding feast , the <unk> of the and wheels , the the against his shoulders . when he swollen blew forty feet and fifth gust of the

In [17]:
torch.manual_seed(SEED); np.random.seed(SEED)
print(beamsearch(lm, text_field, prompt=p, beams=50, max_len=mlen))

the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the southron king - beyond - the - wall had not seemed likely there . their bodies were smashed and <unk> , and free folk were lit in the flowstone rolling - eyes and spears and spears and clams and poles . some of the tents were oats , wheat , the faces entering the field , the maps of gargoyles , and the wedding feast , the cold steel of the wheels , the grey pack of the earth , the swollen rabbit , and the damp stone glow of


# Reference
[1] Speech and language processing: An introduction to natural language processing, computational linguistics, and speech recognition